In [1]:
# build ofa resnet50
from ofa.model_zoo import ofa_net
ofa_network = ofa_net('ofa_resnet50', pretrained=False)


In [2]:
# accuracy predictor
import torch
from ofa.nas.accuracy_predictor import AccuracyPredictor, ResNetArchEncoder
from ofa.utils import download_url

image_size_list = [128, 144, 160, 176, 192, 224, 240, 256]
arch_encoder = ResNetArchEncoder(
	image_size_list=image_size_list, depth_list=ofa_network.depth_list, expand_list=ofa_network.expand_ratio_list,
    width_mult_list=ofa_network.width_mult_list, base_depth_list=ofa_network.BASE_DEPTH_LIST
)

acc_predictor_checkpoint_path = download_url(
    'https://hanlab.mit.edu/files/OnceForAll/tutorial/ofa_resnet50_acc_predictor.pth',
    model_dir='~/.ofa/',
)
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
acc_predictor = AccuracyPredictor(arch_encoder, 400, 3,
                                  checkpoint_path=acc_predictor_checkpoint_path, device=device)

print('The accuracy predictor is ready!')
print(acc_predictor)

Loaded checkpoint from C:\Users\nudtdpj/.ofa/ofa_resnet50_acc_predictor.pth
The accuracy predictor is ready!
AccuracyPredictor(
  (layers): Sequential(
    (0): Sequential(
      (0): Linear(in_features=82, out_features=400, bias=True)
      (1): ReLU(inplace=True)
    )
    (1): Sequential(
      (0): Linear(in_features=400, out_features=400, bias=True)
      (1): ReLU(inplace=True)
    )
    (2): Sequential(
      (0): Linear(in_features=400, out_features=400, bias=True)
      (1): ReLU(inplace=True)
    )
    (3): Linear(in_features=400, out_features=1, bias=False)
  )
)


In [3]:
# build efficiency predictor
from ofa.nas.efficiency_predictor import ResNet50FLOPsModel

efficiency_predictor = ResNet50FLOPsModel(ofa_network)

In [4]:
# search
import random

for i in range(10):
    subnet_config = ofa_network.sample_active_subnet()
    image_size = random.choice(image_size_list)
    subnet_config.update({'image_size': image_size})
    predicted_acc = acc_predictor.predict_acc([subnet_config])
    predicted_efficiency = efficiency_predictor.get_efficiency(subnet_config)

    print(i, '\t', predicted_acc, '\t', '%.1fM MACs' % predicted_efficiency)


0 	 tensor([0.8272], device='cuda:0', grad_fn=<AddBackward0>) 	 2928.9M MACs
1 	 tensor([0.8015], device='cuda:0', grad_fn=<AddBackward0>) 	 2031.2M MACs
2 	 tensor([0.8113], device='cuda:0', grad_fn=<AddBackward0>) 	 1693.6M MACs
3 	 tensor([0.8096], device='cuda:0', grad_fn=<AddBackward0>) 	 1559.9M MACs
4 	 tensor([0.8197], device='cuda:0', grad_fn=<AddBackward0>) 	 2156.3M MACs
5 	 tensor([0.8175], device='cuda:0', grad_fn=<AddBackward0>) 	 2782.9M MACs
6 	 tensor([0.8196], device='cuda:0', grad_fn=<AddBackward0>) 	 3627.9M MACs
7 	 tensor([0.8009], device='cuda:0', grad_fn=<AddBackward0>) 	 2739.5M MACs
8 	 tensor([0.8131], device='cuda:0', grad_fn=<AddBackward0>) 	 2974.9M MACs
9 	 tensor([0.8299], device='cuda:0', grad_fn=<AddBackward0>) 	 4968.8M MACs
